**PAQUETERIAS NECESARIAS**

In [11]:
import pickle
 
pickled_file = open('pickled_file.pickle', 'rb')
 
name = pickle.load(pickled_file)
website = pickle.load(pickled_file)
english_french = pickle.load(pickled_file)
tup = pickle.load(pickled_file)
 
print('Name: ')
print(name)
print('Website:')
print(website)
print('Englsh to French:')
print(english_french)


EOFError: Ran out of input

In [1]:
## INCLUYE TODOS LOS PAQUETES A USAR
import tensorflow as tf
import random
from tensorflow import keras
 # Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import gaussian
 # Biblioteca para datos de entrenamiento
from sklearn.model_selection import train_test_split
import h5py as h # manipular extensiones
# FUNCIONES DEFINIDAS PARA USO NECESARIO
import timeit

**ENTRADA DE LA INFORMACION, ORDENAMIENTO DE LA INFORMACION A USAR**

In [2]:
 # ENTRAR LA INFORMACION DESDE UN ARCHIVO EXTERNO
FILE  = h.File("DATOS/TODOS.h5", 'r')
INPUT = np.vstack([FILE["qcd/images"][:], FILE["wprime/images"][:]]); INPUT=INPUT/np.max(INPUT) # SE NORMALIZA
LABEL = np.hstack([FILE["qcd/labels"][:], FILE["wprime/labels"][:]])
OUTPUT_LINEAL = np.ones((len(LABEL), 2));
OUTPUT_LINEAL[:,0] = LABEL < 1    ;     OUTPUT_LINEAL[:,1] = LABEL > 0
 # REACONDICIONAR LA INFORMACION INPUT EN UNA MATRIX PARA SU RAPIDO ACCESO
INPUT_LINEAL = np.reshape(INPUT, [len(INPUT),-1]) # LA INFORMACION SE LINEALIZA Y SE LOCALIZA EN LAS FILAS
 # SEPARAR LA INFO PARA ENTRENAR DE LA QUE SERA USADA PARA EL TEST DE CORRESPONDENCIA
INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL, OUTPUT_LINEAL_TEST = train_test_split( 
    INPUT_LINEAL, OUTPUT_LINEAL, test_size=0.2, shuffle = True);

**FUNCIONES INICIALES Y NECESARIAS PARA EL ENTRENAMIENTO**

In [3]:
## CLASE DE LOS VALORES A TENER EN CUENTA
'''class arreglo():
    def __init__( self , v_input ):#, exp = 1 , cross_entropy = 1, n_iterations = 3000, dropout = 0.5):
        self.n_capas  = 2;         self.v_output = 2;         self.v_input = v_input 
        self.exp      = 1;
        #self.matrix_neuronas = {}
        self.learning_rate = 1e-1; self.cross_entropy = 1;    self.n_iterations  = 3000;
        self.dropout       = 0.5;
        self.n_step_visual_process = 10;'''
## CLASE DE LOS VALORES A TENER EN CUENTA
config = tf.compat.v1.placeholder("float")
config.v_input= len(INPUT_LINEAL[0,:]); config.v_output = len(OUTPUT_LINEAL[0,:]);#, exp = 1 , cross_entropy = 1, n_iterations = 3000, dropout = 0.5):
config.n_capas  = 2;                    config.exp      = 1;
#self.matrix_neuronas = {} # SE RELLENARA EN EL FUTURO
config.learning_rate = 1e-1; config.cross_entropy = 1;    config.n_iterations  = 3000;
config.dropout       = 0.5;
config.n_step_visual_process = 10;
## NO DEBEN VOLVERSE A DEFINIR BAJO NINGUN CONCEPTO
config.X = tf.compat.v1.placeholder("float", shape = [None,len(INPUT_LINEAL[0,:])] ) #, name= "input_size2"); # Estructura de la entrada
config.Y = tf.compat.v1.placeholder("float", shape = [None,len(OUTPUT_LINEAL[0,:])] ) #, name= "output_size2") #Estructura de la salida
    

In [4]:
## DEFINICION DE LOS MODOS DE ELECCION DE CAPAS OCULTAS Y NUMEROS DE NEURONAS
def neuronas( config ): #v_input, v_output, n_capas = 2, exp = 1,  modo = 'poly' ):
    #config = Dd()
    x = np.ones(config.n_capas + 2)
    #print(x)
    if  config.exp >= 1: #modo == 'poly':
        for i in range(config.n_capas + 2):
            x[len(x)-i-1] = round(( config.v_input - config.v_output )*pow(i/(config.n_capas + 1 ), config.exp) + config.v_output)
            #print(x[i])
            #print(i)
    elif config.exp < 1:
        for i in range(config.n_capas + 2):
            x[len(x)-i-1] = round(( config.v_input - config.v_output )*pow(i/( config.n_capas + 1 ), -1/config.exp) + config.v_output)
            #print(x[i])
            #print(i)
    config.matrix_neuronas = np.int64(x)
    return config

In [5]:
config = neuronas( config ) 

In [6]:
print(config.cross_entropy)

1


In [7]:
## ORGANIZACION DE LA ESTRUCTURA INTERNA DE ENTRENAMIENTO ##
def inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , config):
    config = neuronas( config )
    #proceso iteractivo dinamico 
    for i in range(len(config.matrix_neuronas)-1):

        weights = tf.Variable( tf.truncated_normal([config.matrix_neuronas[i], config.matrix_neuronas[i+1]], stddev=0.1) , 
                              name = f"weights-w{i}")
        biases = tf.Variable(tf.constant(0.1, shape=[config.matrix_neuronas[i+1]]) , name=f"biases_b{i}" )
        
        if  i == 0: # CONDICION INICIAL
            layer = tf.add(tf.matmul( config.X, weights), biases , name=f"layer_l{i}" )
        else: # VALORES INTERNMEDIOS
            layer = tf.add(tf.matmul(    layer, weights), biases , name=f"layer_l{i}" )
    # PARAMETROS FINALES PARA CARACTERIZACION 
    config.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = config.Y, logits=layer) , 
                                          name= "Definition_of_cross_entropy")
    config.train_step    = tf.compat.v1.train.AdamOptimizer( config.learning_rate ).minimize( config.cross_entropy , 
                                          name= "Definition_of_train_step")
    config.correct_pred  = tf.equal(tf.argmax(layer, 1), tf.argmax( config.Y , 1) , 
                                          name= "Definition_of_correct_pred")
    config.accuracy      = tf.reduce_mean(tf.cast( config.correct_pred  , tf.float32) ,
                                          name= "Definition_of_accuracy")
#    return config
#def inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , config):
    
    init = tf.compat.v1.global_variables_initializer()

    config.cost_summary = tf.compat.v1.summary.scalar("Cost", config.cross_entropy)
    config.acc_summary = tf.compat.v1.summary.scalar("Accuracy", config.accuracy)
    config.all_summary = tf.compat.v1.summary.merge_all()#'''
    
    
    #saver=tf.train.Saver() #SALVAR LA SECCION PARA PODERLA UTILIZAR LUEGO
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("Tensorboard", sess.graph)
        #print(sess)
        sess.run(init)
        #saver.save(sess, 'modelo')
        toc_general = 0  
        for i in range(config.n_iterations):
            tic = timeit.default_timer()
            sess.run(config.train_step, feed_dict={ config.X: INPUT_LINEAL, config.Y:OUTPUT_LINEAL}) # CORRIDA

            config.summary_results, config.loss, config.acc = sess.run([config.all_summary, config.cross_entropy, config.accuracy], 
                                                  feed_dict={config.X: INPUT_LINEAL, config.Y: OUTPUT_LINEAL})
            writer.add_summary(config.summary_results, i)
            
            toc = timeit.default_timer() ;   toc_general += toc ;
            if (i)%10 == 0:
                print("Iteration", str(i), "\t| Loss =", str( config.loss ), "\t| Accuracy =", str( config.acc )
                     , "\t| Time =", str( toc - tic ))
            
            if config.acc>.9:
                print("Iteration", str(i), "\t| Loss =", str( config.loss ), "\t| Accuracy =", str( config.acc ))
                break
        config.test_accuracy = sess.run(config.accuracy, feed_dict={config.X: INPUT_LINEAL_TEST, config.Y: OUTPUT_LINEAL_TEST})
        print("\n Accuracy on test set:", config.test_accuracy , "\n Mean time of Iterations:", str(toc_general/i) )

        #a, b = sess.run([tf.argmax(layer, 1),tf.argmax(Y,1)], feed_dict={X: INPUT_LINEAL, Y: OUTPUT_LINEAL}) 

        #for ea, eb in zip(a, b):
        #    print(ea,eb)
    #return 0


**INICIALIZAR EL ENTRENAMIENTO**

In [8]:
#config = neuronas( config )
inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , config )

Iteration 0 	| Loss = 42.924084 	| Accuracy = 0.5969325 	| Time = 0.6084820999999998
Iteration 10 	| Loss = 2.690886 	| Accuracy = 0.26617128 	| Time = 0.2924161999999999
Iteration 20 	| Loss = 11.881383 	| Accuracy = 0.46403736 	| Time = 0.26037900000000036
Iteration 30 	| Loss = 6.6181126 	| Accuracy = 0.48690102 	| Time = 0.2839989000000003
Iteration 40 	| Loss = 1.4642833 	| Accuracy = 0.877203 	| Time = 0.31932079999999985
Iteration 50 	| Loss = 3.6809416 	| Accuracy = 0.83814424 	| Time = 0.30093670000000117
Iteration 60 	| Loss = 1.3226497 	| Accuracy = 0.88425267 	| Time = 0.2697593000000005
Iteration 64 	| Loss = 0.44879648 	| Accuracy = 0.901972

 Accuracy on test set: 0.89485717 
 Mean time of Iterations: 21.406706671875


In [9]:
'''#proceso iteractivo dinamico 
for i in range(len(config.matrix_neuronas)-1):
    weights = tf.Variable( tf.truncated_normal([config.matrix_neuronas[i], config.matrix_neuronas[i+1]], stddev=0.1) 
                            , name = f"weights-w{i}")
    biases = tf.Variable(tf.constant(0.1, shape=[config.matrix_neuronas[i+1]]) 
                            , name=f"biases_b{i}" )
    
    if  i == 0: # CONDICION INICIAL
        layer = tf.add(tf.matmul(config.X, weights), biases , name=f"layer_l{i}" )
    else: # VALORES INTERNMEDIOS
        layer = tf.add(tf.matmul(layer   , weights), biases , name=f"layer_l{i}" )
    
# PARAMETROS FINALES PARA CARACTERIZACION 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = config.Y, logits=layer) )
train_step    = tf.compat.v1.train.AdamOptimizer( config.learning_rate ).minimize( cross_entropy )
correct_pred  = tf.equal(tf.argmax(layer, 1), tf.argmax( config.Y , 1) )
accuracy      = tf.reduce_mean(tf.cast( correct_pred  , tf.float32))#,

init = tf.compat.v1.global_variables_initializer()

cost_summary = tf.compat.v1.summary.scalar("Cost", config.cross_entropy)
acc_summary = tf.compat.v1.summary.scalar("Accuracy", accuracy)
all_summary = tf.compat.v1.summary.merge_all()#

#saver=tf.train.Saver() #SALVAR LA SECCION PARA PODERLA UTILIZAR LUEGO
with tf.Session() as sess:
    writer = tf.summary.FileWriter("Tensorboard", sess.graph)
    sess.run(init)
    for i in range(config.n_iterations):
        sess.run( train_step, feed_dict={config.X: INPUT_LINEAL, config.Y:OUTPUT_LINEAL})  # CORRIDA
        #summary_results, loss, acc = sess.run( [all_summary, config.cross_entropy, accuracy], 
        #                                       feed_dict={X: INPUT_LINEAL, Y: OUTPUT_LINEAL} )
        summary_results, loss, acc = sess.run([all_summary, cross_entropy, accuracy], 
                                              feed_dict={config.X: INPUT_LINEAL, config.Y: OUTPUT_LINEAL})
        writer.add_summary(summary_results, i)
        if (i)%10 == 0:
            print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))

        if acc>.70:
            print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
            break
    test_accuracy = sess.run(accuracy, feed_dict={config.X: INPUT_LINEAL_TEST, config.Y: OUTPUT_LINEAL_TEST})
    print("\nAccuracy on test set:", test_accuracy)'''

Iteration 0 	| Loss = 42.593365 	| Accuracy = 0.5969325
Iteration 7 	| Loss = 1.9583709 	| Accuracy = 0.839859

Accuracy on test set: 0.8445714


**PAQUETERIAS NECESARIAS**

In [10]:
inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , config )

Iteration 0 	| Loss = 15.195285 	| Accuracy = 0.5969325 	| Time = 0.7342865999999972
Iteration 10 	| Loss = 15.678536 	| Accuracy = 0.7360198 	| Time = 0.49411649999999696
Iteration 20 	| Loss = 10.265811 	| Accuracy = 0.450986 	| Time = 0.42484879999999947
Iteration 30 	| Loss = 5.3215785 	| Accuracy = 0.7708869 	| Time = 0.41935369999999494
Iteration 40 	| Loss = 2.76392 	| Accuracy = 0.85681623 	| Time = 0.5196905000000029
Iteration 49 	| Loss = 0.6513335 	| Accuracy = 0.90806895

 Accuracy on test set: 0.9047619 
 Mean time of Iterations: 50.20468473877552


In [11]:
print(config.matrix_neuronas[-1] )
print( config.n_capas )
print( config.v_output )
print( config.v_input )
print( config.learning_rate )
print( config.cross_entropy )
print( config.n_iterations )     
print( config.dropout )
print( config.n_step_visual_process )
print( config.v_input )
print( len(INPUT_LINEAL[0,:]) )
print( len(OUTPUT_LINEAL[0,:]) )
print( INPUT_LINEAL.shape )
print( OUTPUT_LINEAL.shape )
print( X.shape )
print( Y.shape )
print( config.train_step )
print( config.accuracy)

2
2
2
625
0.1
Tensor("Definition_of_cross_entropy_1:0", shape=(), dtype=float32)
3000
0.5
10
625
625
2
(10497, 625)
(10497, 2)


NameError: name 'X' is not defined

In [ ]:
'''def inicio( INPUT_LINEAL, INPUT_LINEAL_TEST, OUTPUT_LINEAL , OUTPUT_LINEAL_TEST , CONFIG):   
    cost_summary  = tf.compat.v1.summary.scalar("Cost", config.cross_entropy)
    acc_summary   = tf.compat.v1.summary.scalar("Accuracy", config.accuracy)
    all_summary   = tf.compat.v1.summary.merge_all()
    # COMENZAR EL PROCESO DE ENTRENAMIENTO
    X = tf.compat.v1.placeholder("float", shape = [None,len(INPUT_LINEAL[0,:])] , name= "input_size"); # Estructura de la entrada
    Y = tf.compat.v1.placeholder("float", shape = [None,len(OUTPUT_LINEAL[0,:])] , name= "output_size") #Estructura de la salida
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("Tensorboard", sess.graph)
        sess.run(init)
        #toc_general=0; # TIEMPO DE TODO EL PROCESO ITERATIVO
        for i in range(CONFIG.n_iterations):
            #tic = 1#timeit.default_timer(); # COMENZAR EL CONTEO DEL TIEMPO DEL PROCESO
            sess.run(CONFIG.train_step, feed_dict={X:INPUT_LINEAL , Y:OUTPUT_LINEAL}) # CORRIDA

            summary_results, loss, acc = sess.run([all_summary,  CONFIG.cross_entropy, CONFIG.accuracy], 
            #                                                           feed_dict = {X: INPUT_LINEAL, Y: OUTPUT_LINEAL} )
            writer.add_summary(summary_results, i)
            #toc = 2#timeit.default_timer(); # CORTE FINAL DEL TIEMPO PARA IMPRESION
            if  (i)%100 == 0:
                print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
    
            if  acc>.70:
                print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
                break
            #toc_general += tic-toc;        
        CONFIG.test_accuracy = sess.run(CONFIG.accuracy, feed_dict={X: INPUT_LINEAL_TEST, Y: OUTPUT_LINEAL_TEST})
        print("\nAccuracy on test set:", CONFIG.test_accuracy)
    
    return CONFIG'''

In [ ]:
# SE REALIZA LA OPERACION UNA SOLA VEZ
#print(len(INPUT_LINEAL[0,:]))
#config = arreglo( v_input = len(INPUT_LINEAL[0,:]) )

#config = neuronas( config )
print(config.matrix_neuronas)
#print(config.matrix_neuronas)
#config = estructura( INPUT_LINEAL, OUTPUT_LINEAL , config )
#X = tf.compat.v1.placeholder("float", shape=[ None, len(INPUT_LINEAL[0,:]) ] , name= "input_size00"); # Estructura de la entrada
#Y = tf.compat.v1.placeholder("float", shape=[None, len(OUTPUT_LINEAL[0,:]) ] , name= "output_size00") #Estructura de la salida
